In [ ]:
# 조이스틱 : 젯봇 -> 본체로 옮기기

### 무선으로 조이스틱을 활용해보자! 
- 젯봇 조종, 카메라도 같이 활용
- https://hardwaretester.com/gamepad 사이트 접속
- 사이트에서 확인한 인덱스는 0

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
controller = widgets.Controller(index = 0)
display(controller)

Controller()

In [38]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

import traitlets # 속성을 관리하는 라이브러리 -> 카메라가 위젯과 연결될 때 데이터가 빠르게 바뀔 때마다 빠르게 적용시켜줌
                 # 2개의 값을 Link시켜주는 라이브러리
    
camera = Camera.instance(width = 224, height = 224)  # 실제 사용할 카메라
image = widgets.Image(width = 224, height = 224, format = 'jpeg')  # 위젯에서 정해주는 카메라

# digital link -> camera의 value와 image의 value를 link시키자
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform = bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [33]:
# 카메라 끄기
camera_link.unlink()

In [ ]:
# 조이스틱 조종, 카메라 활용!
# 두 가지 일을 동시에 처리해야함!

In [10]:
# 숫자를 출력하는 함수!
import time
def count_number(start, end) :
    for i in range(start, end+1):
        print(i, end = " ")
        time.sleep(0.5)
    print()

In [11]:
count_number(1, 10)
count_number(101, 110)

1 2 3 4 5 6 7 8 9 10 
101 102 103 104 105 106 107 108 109 110 


In [ ]:
# 함수는 동시에 일을 하지 않는다. 한 함수가 끝나야 다음 함수가 실행됨!
# -> 두 가지 일을 동시에 처리해야 한다면?
# -> Thread 라이브러리 사용!

### Thread 라이브러리 활용하기!
- threading을 import 
- threading.Thread(target = 사용할함수, args = (범위지정))

In [14]:
import threading
thread1 = threading.Thread(target = count_number, args = (1, 30))
thread2 = threading.Thread(target = count_number, args = (10, 20))
thread1.start()
thread2.start()

1 10 2 11 3 12 4 13 5 14 6 15 7 16 8 17 9 18 10 19 11 20 12 
13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 


In [15]:
# 외부에서 Thread 능동적으로 멈추는 함수 정의
import inspect # inspect -> 코드 내의 객체에 대한 정보를 검사하는 데 사용
import ctypes # C언어 타입 / ctypes 모듈은 C 라이브러리를 사용하기 위한 외부 함수 호출을 제공하며, 이 때 C 타입을 다루기 위해 사용 

# stop_thread(멈출 쓰레드)
# 이 함수를 실행하면 쓰레드가 멈춘다!!

def _async_raise(tid, exctype): #쓰레드에게 예외를 발생시키는 역할
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid) #쓰레드 식별자를 C 타입인 c_long으로 변환
    if not inspect.isclass(exctype): #exctype가 클래스가 아니면
        exctype = type(exctype) # 해당 객체의 타입을 가져와서 클래스로 사용
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype)) #함수를 사용하여 쓰레드에게 예외를 발생 -> 반환값은 성공 여부
    if res == 0:#반환값이 0이면
        raise ValueError("invalid thread id") #쓰레드 식별자가 유효하지 않다는 예외를 발생
    elif res != 1:#반환값이 1이 아니면 
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None) #쓰레드에게 예외를 다시 발생시켜서 효과를 되돌림
        
def stop_thread(thread): # 주어진 쓰레드를 강제로 종료 
    _async_raise(thread.ident, SystemExit) #SystemExit 예외를 발생시킴으로써 쓰레드를 종료

In [18]:
import threading
thread1 = threading.Thread(target = count_number, args = (1, 100))
thread2 = threading.Thread(target = count_number, args = (10, 100))
thread1.start()
thread2.start()

1 10 2 11 3 12 413  14 5 15 6 16 7 17 8 18 9 19 10 

In [19]:
stop_thread(thread1)
stop_thread(thread2)

In [39]:
import custom
import head 
def jetbot_motion():
    while 1: 
        # [1] -> 왼쪽 조이스틱 위아래
        if controller.axes[1].value == 1: # 왼쪽 조이스틱 아래
            custom.backward()
        elif controller.axes[1].value == -1: # 왼쪽 조이스틱 위
            custom.forward()

        if controller.buttons[0] == True:  # 1번 버튼 눌렀을 때
            custom.stop()
        if controller.buttons[1] == True : # 2번 버튼 눌렀을 때 
            head.center()
        if controller.buttons[2] == True : # 3번 버튼 눌렀을 때
            head.tilt_down()
        if controller.buttons[3] == True : # 4번 버튼 눌렀을 때
            head.tilt_up()

In [40]:
import threading
thread3 = threading.Thread(target = jetbot_motion)
thread3.start()

In [41]:
stop_thread(thread3)

In [31]:
custom.stop()